# Summarize the paper as a tree
In this example we read the labeled paper as a tree.
Then we summarize it by using the LlamaIndex TreeSummarize function.

We can either use OpenAI's API or load a model to do it locally.

## Load LLM and TreeSummarize

In [1]:
from llama_index.response_synthesizers import TreeSummarize
from llama_index import SimpleDirectoryReader, ServiceContext
import os
os.chdir('..')
#import openai
#openai.api_key = "your key"

import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="StabilityAI/stablelm-tuned-alpha-3b",
    model_name="StabilityAI/stablelm-tuned-alpha-3b",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)
service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm)
summarizer = TreeSummarize(service_context=service_context, verbose=True)

/home/duan/texcompile/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.36s/it]


******
Could not load OpenAIEmbedding. Using HuggingFaceBgeEmbeddings with model_name=BAAI/bge-small-en. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

******


## Load data from exported csv

In [2]:
import pandas as pd
df = pd.read_csv('text.csv', sep='\t')
text = ''
section_id = 0
for i, row in df[df['reading_order']!=-1].sort_values(by=['reading_order', 'Unnamed: 0']).iterrows():
    if row['section_id'] > section_id:
        text += '\n'
        section_id = row['section_id']
    if row['label'] != 'Figure':
        text += row['token'] + ' '

## Get response

In [15]:
sections =  text.split('\n')
response = summarizer.get_response("what are this texts talking about, emphasize the second section:", sections)
response2 = summarizer.get_response("what are this texts talking about, emphasize the evaluation section:", sections)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


1 text chunks after repacking


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


1 text chunks after repacking


In [13]:
response

'This text is discussing the use of Convolutional Neural Networks (CNNs) for automatic crater detection. Crater detection is a crucial step in the study of the Solar System, as it provides valuable information about the past and present geological processes. CNNs have been used successfully in the past to detect craters, but they have not yet been widely adopted for automatic crater detection. The authors present a method for automatic crater detection using CNNs, which are organized as a computation graph. The CNN architecture is a crucial component of the computation graph, as it allows for the efficient processing of large amounts of data. The authors also discuss the use of CNNs for detecting craters and the challenges they face in automatically detecting craters. The text highlights the importance of using CNNs for automatic crater detection and the potential benefits of using them for this purpose.'

In [16]:
response2

'This text is discussing the Crater Detection via Convolutional Neural Networks (CNN) approach, which uses advanced machine learning techniques to automatically detect craters. The text highlights the challenges of automatically detecting craters, such as the complex surface of craters, and the need for advanced machine learning techniques to deal with the large amount of satellite imagery. The text also mentions the use of Convolutional Neural Networks (CNN) for automatic crater detection, which is a significant breakthrough in the field. The text also mentions the use of CNNs for automatic crater detection, which is a new approach that has achieved good performance on this dataset. The text also mentions the use of Banderia, a dataset that contains high-quality craters across all tiles, and the evaluation of the CNN approach using the Banderia dataset. The text concludes by discussing the potential of CNNs for automatic crater detection and the need for further advancements in the fi